# NanoServices: SSH-Key pro Repo festlegen (GitHub) – Schritt für Schritt
Dieses Notebook zeigt **einfach und reproduzierbar**, wie du deinen **SSH-Key** für ein konkretes Git-Repository festlegst, prüfst und testest.

**Beispiel:** `~/dev/nanoservices/java/nanoservices-temporal` mit Remote `git@github.com:nanoservices-dev/nanoservices-temporal.git`


✅ **Bekannt funktionierender Key (aus deiner Session):**

- **Pfad:** `~/.ssh/id_github_nanoservices`

- **Typ:** ED25519

- **GitHub-Account:** `GeorgeTheDragon`

- **Nachricht bei Test:** `Hi GeorgeTheDragon! You've successfully authenticated, but GitHub does not provide shell access.` (normal)


> **Hinweis:** GitHub bietet keine Shell – die Meldung ist ein erfolgreicher Auth-Check, kein Fehler.


## 0) Kontext prüfen (Repo & Remote)
Im Terminal **im Repo** ausführen (hier: `~/dev/nanoservices/java/nanoservices-temporal`).

In [ ]:
# (Shell) Remote anzeigen
# Im Terminal ausführen:
#   cd ~/dev/nanoservices/java/nanoservices-temporal
#   git remote -v
print("Im Terminal ausführen:\n  cd ~/dev/nanoservices/java/nanoservices-temporal\n  git remote -v")

## 1) Welche SSH-Public-Keys existieren lokal?
Liste deine öffentlichen Schlüssel auf. Die passenden **Private Keys** haben denselben Namen ohne `.pub`.


In [ ]:
# (Shell) Public Keys anzeigen
# macOS/Linux:
#   ls -l ~/.ssh/*.pub
print("Im Terminal ausführen:\n  ls -l ~/.ssh/*.pub")

## 2) Welcher Key gehört zu welchem GitHub-Account?
**Test pro Key** (hier exemplarisch für die wichtigsten Dateien). Ein erfolgreicher Test zeigt `Hi <USERNAME>!` und ist **kein** Shell-Login, sondern nur Auth-Check.


In [ ]:
# (Shell) Jeden Key explizit testen – einer nach dem anderen
#   ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_github_nanoservices git@github.com
#   ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_ed25519           git@github.com
#   ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_rsa               git@github.com
#   ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_nanoservices_dev  git@github.com
print("Im Terminal (jeweils einzeln) ausführen:\n"
      "  ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_github_nanoservices git@github.com\n"
      "  ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_ed25519           git@github.com\n"
      "  ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_rsa               git@github.com\n"
      "  ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_nanoservices_dev  git@github.com")

**Erwartung für den _funktionierenden_ Key in deinem Setup:**  
- „**Server accepts key** … `/Users/christianfinkbeiner/.ssh/id_github_nanoservices` …“  
- „**Hi `GeorgeTheDragon`!** You've successfully authenticated, but GitHub does not provide shell access.“

## 3) Diesen Key **repo-spezifisch** fest verdrahten
So erzwingst du, dass **dieses Repo** _immer_ den Key `~/.ssh/id_github_nanoservices` nimmt – unabhängig von Agent/Global-Config.


In [ ]:
# (Shell) Im Repo ausführen
#   cd ~/dev/nanoservices/java/nanoservices-temporal
#   git config core.sshCommand 'ssh -i ~/.ssh/id_github_nanoservices -o IdentitiesOnly=yes'
print("Im Terminal ausführen:\n  cd ~/dev/nanoservices/java/nanoservices-temporal\n"
      "  git config core.sshCommand 'ssh -i ~/.ssh/id_github_nanoservices -o IdentitiesOnly=yes'")

## 4) Sicherstellen, dass der Remote **SSH** ist (nicht HTTPS)
Falls noch HTTPS eingetragen ist, auf SSH-URL umstellen.


In [ ]:
# (Shell) Remote fest auf SSH setzen (konkrete URL)
#   git remote set-url origin git@github.com:nanoservices-dev/nanoservices-temporal.git
print("Im Terminal ausführen:\n  git remote set-url origin git@github.com:nanoservices-dev/nanoservices-temporal.git")

## 5) Zugriff prüfen (ohne Push)
`git ls-remote` fragt die Referenzen ab – klappt nur mit gültigem Zugriff.


In [ ]:
# (Shell) Test – muss ohne Permission denied zurückkommen
#   git ls-remote origin
print("Im Terminal ausführen:\n  git ls-remote origin")

## 6) (Wichtig für Organisations-Repos) **SSO freischalten**
Wenn dein Schlüssel unter **GitHub → Settings → SSH and GPG keys** hinterlegt ist, musst du bei **Organisationen** (hier: `nanoservices-dev`) oft zusätzlich **„Enable SSO“** für diesen Key klicken – sonst scheitern private Org-Repos mit `Permission denied (publickey)`.


## 7) Troubleshooting – häufige Fälle
- **`Permission denied (publickey)`** trotz korrektem Key:

  1. Public Key nicht bei GitHub hinterlegt → `pbcopy < ~/.ssh/id_github_nanoservices.pub` und auf GitHub einfügen.

  2. SSO für die Organisation nicht aktiviert → „Enable SSO“ beim Key anklicken.

  3. Falscher Key im Repo → `git config core.sshCommand` prüfen.

  4. Falsche Remote-URL (HTTPS statt SSH) → `git remote -v` prüfen und ggf. umstellen (siehe oben).


- **Welcher Key wird wirklich benutzt?**

  ```bash
  ssh -T -v -o IdentitiesOnly=yes -i ~/.ssh/id_github_nanoservices git@github.com
  ```
  Achte auf „**Server accepts key:** …`id_github_nanoservices`…“ und die `Hi <USERNAME>!`-Zeile.

- **Repo temporär mit bestimmtem Key testen (ohne Config zu ändern):**

  ```bash
  GIT_SSH_COMMAND='ssh -i ~/.ssh/id_github_nanoservices -o IdentitiesOnly=yes' git ls-remote origin
  ```


---
### Zusammenfassung
- **Key:** `~/.ssh/id_github_nanoservices` (**funktioniert**, Account: `GeorgeTheDragon`).  
- **Repo:** `~/dev/nanoservices/java/nanoservices-temporal`  
- **Remote:** `git@github.com:nanoservices-dev/nanoservices-temporal.git`  
- **Repo bindet Key:**  
  ```bash
  git config core.sshCommand 'ssh -i ~/.ssh/id_github_nanoservices -o IdentitiesOnly=yes'
  ```
- **Zugriffstest:**  
  ```bash
  git ls-remote origin
  ```
